# Error mitigation

This notebook contains theoretical background and examples for using our Error Mitigation module with Qiskit.

## Theoretical background
Here we describe main theoretical concepts related to mitigation procedure. For more detailed description, see Ref. [0].

### Classical noise model
Let us denote by $M$ a POVM describing noisy detector and $P$ denote ideal measurement. In classical noise model, we assume that the relation between the two is given by stochastic, invertible map $\Lambda$:

$$ M = \Lambda P.$$

Let us denote by $p_{exp}$ vector of probabilities obtained on a noisy detector $M$ measuring any quantum state, and by $p_{ideal}$ the analogous vector for ideal detector (for the same quantum state). From linearity of the Born's rule, it follows that two vectors are related by the same stochastic map as POVMs:

$$ p_{exp} = \Lambda p_{ideal}. $$

Recall that we assumed that $\Lambda$ is invertible. Hence by multiplying last equation by $\Lambda^{-1}$ from both sides, we obtain

$$ \Lambda^{-1} p_{exp} = p_{ideal}. $$

Effectively, by this kind of postprocessing, we obtain statistics which we would have obtained on the perfect detector devices.

### Deviations from noise model


### Finite-size statistics

### When is mitigation succesfull?






#### References
[0] Filip B. Maciejewski, Zoltán Zimborás, Michał Oszmaniec, "Mitigation of readout noise in near-term quantum devices by classical post-processing based on detector tomography", arxiv preprint, https://arxiv.org/abs/1907.08518 (2019)
  



## Mitigating error using our module

### Performing Quantum Detector Tomography
Our error mitigation approach is based on the knowledge about the noise in the device's detector. Such knowledge can be obtained in procedure known as Quantum Detector Tomography (QDT). To perform QDT, one can follow the steps from our [QDT tutorial](https://github.com/fbm2718/QREM/blob/master/QDT_Tutorial.ipynb). 


In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter


# Choose qubit indices
QDT_qubit_index = [3]

# Select probe kets
QDT_probe_kets = povmtools.pauli_probe_eigenkets

# Generate circuits
QDT_circuits = detector_tomography_circuits(QDT_qubit_index, QDT_probe_kets)

# Get QDT circuits results
backend = Aer.get_backend('qasm_simulator')  #  Get backed
shots_number = 2000  # Define number of measurement repetitions
QDT_job = execute(QDT_circuits, backend=backend, shots=shots_number)
results = QDT_job.result()

# Get ml_povm_estimator using DTF and results
DTF = DetectorTomographyFitter()
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], QDT_probe_kets)

## Preparing mitigation

Now that we have the estimator of POVM, we can create QDTErrorMitigator object and prepare it.

In [ ]:
# Creation and preparation of QDTErrorMitigator
mitigator = QDTErrorMitigator()
mitigator.prepare_mitigator(ml_povm_estimator)

With prepared mitigator object we gain access to several useful functionalities. For example, we can:
* Correct results of qiskit job by using apply_correction_to_qiskit_job(Results) method.
* Access transition and correction matrices obtained from POVM given during preparation.

In order to properly analyse the results of correction procedure, one have to be aware that in some cases raw application of $\Lambda^{-1}$ to the results may yield quasiprobability (instead of probability) vectors. In such scenario our method calculates closest probability vectors and returns them instead. Distances from raw quasiprobabilities to returned probabilities, can be accessed via distances_from_closest_probability_vector member of mitigator object.

### Error bounds

With access to POVM and the correction and transition matrices, we are able to calculate bounds on several errors. In particular, using povtools module, we can calculate:
*statistical error bound (using get_statistical_erro, coherent error bound and correction error bound.  